In [23]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

In [26]:
from sklearn.model_selection import train_test_split
import pandas as pd

# CSV 파일 로드 함수
def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data

# 데이터 로드
data_bumper = load_data_with_pandas('./data/data/output/bumper/bumper_data.csv')
data_door_ed = load_data_with_pandas('./data/data/output/door/door_ed_data.csv')
data_door_scratch = load_data_with_pandas('./data/data/output/door/door_scratch_data.csv')
data_fender = load_data_with_pandas('./data/data/output/fender/fender_ed_data.csv')
data_frame_ed = load_data_with_pandas('./data/data/output/frame/frame_ed_data.csv')
data_frame_hd = load_data_with_pandas('./data/data/output/frame/frame_hd_data.csv')
data_frame_sealf = load_data_with_pandas('./data/data/output/frame/frame_sealf_data.csv')
data_frame_seamf = load_data_with_pandas('./data/data/output/frame/frame_seamf_data.csv')


# 0이 처음 출력되는 index 값 추출
first_zero_bumper = data_bumper[data_bumper['quality'] == 0].index[0]

first_zero_door_ed = data_door_ed[data_door_ed['quality'] == 0].index[0]
first_zero_door_scratch = data_door_scratch[data_door_scratch['quality'] == 0].index[0]

first_zero_fender = data_fender[data_fender['quality'] == 0].index[0]

first_zero_frame_ed = data_frame_ed[data_frame_ed['quality'] == 0].index[0]
first_zero_frame_hd = data_frame_hd[data_frame_hd['quality'] == 0].index[0]
first_zero_frame_sealf = data_frame_sealf[data_frame_sealf['quality'] == 0].index[0]
first_zero_frame_seamf = data_frame_seamf[data_frame_seamf['quality'] == 0].index[0]

In [27]:
import pandas as pd

def modify_csv(input_file, output_file, column_name, new_value, first_zero_frame_hd):
    # CSV 파일 읽기
    df = pd.read_csv(input_file)

    # 조건 설정: 첫 번째 0이 나오기 전까지만
    row_condition = df.index < first_zero_frame_hd

    # 조건에 해당하는 값 수정
    df.loc[row_condition, column_name] = new_value

    # 수정된 데이터 저장
    df.to_csv(output_file, index=False)

modify_csv(
    "./data/data/output/frame/frame_hd_data.csv", 
    "./data/data/output/frame/frame_hd_data_modify.csv", 
    "bboxes", 
    "[]", 
    first_zero_frame_hd
)

modify_csv(
    "./data/data/output/frame/frame_sealf_data.csv", 
    "./data/data/output/frame/frame_sealf_data_modify.csv", 
    "bboxes", 
    "[]", 
    first_zero_frame_sealf
)

modify_csv(
    "./data/data/output/frame/frame_seamf_data.csv", 
    "./data/data/output/frame/frame_seamf_data_modify.csv", 
    "bboxes", 
    "[]", 
    first_zero_frame_seamf
)

In [28]:
import pandas as pd
data_door = pd.concat([data_door_ed.iloc[:first_zero_door_ed],
                      data_door_scratch.iloc[:first_zero_door_scratch]], ignore_index=True)
data_frame = pd.concat([data_frame_ed.iloc[:first_zero_frame_ed],
                        data_frame_hd.iloc[:first_zero_frame_hd], data_frame_sealf.iloc[:first_zero_frame_sealf], data_frame_seamf.iloc[:first_zero_frame_seamf]], ignore_index=True)

bumper_before_zero = data_bumper.iloc[:first_zero_bumper]
fender_before_zero = data_fender.iloc[:first_zero_fender]


# 75개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_ok_random_sample = bumper_before_zero.sample(
    n=min(75, len(bumper_before_zero)), replace=False, random_state=55)
fender_ok_random_sample = fender_before_zero.sample(
    n=min(75, len(fender_before_zero)), replace=False, random_state=55)
door_ok_random_sample = data_door.sample(n=75, replace=False)
frame_ok_random_sample = data_frame.sample(n=75, replace=False)
# 결과 출력
# print(bumper_ok_random_sample)
# print(len(bumper_ok_random_sample))

# print(fender_ok_random_sample)
# print(len(fender_ok_random_sample))

# print(door_ok_random_sample)
# print(len(door_ok_random_sample))

# print(frame_ok_random_sample)
# print(len(frame_ok_random_sample))

ok_data = pd.concat([bumper_ok_random_sample, fender_ok_random_sample,
                    door_ok_random_sample, frame_ok_random_sample], ignore_index=True, axis=0)

ok_data

,path,quality,part,bboxes
0,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
1,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
2,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
3,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
4,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
...,...,...,...,...
295,./data/data/source_data/frame/Sealing_failure\...,1,프레임,"[[1415.7093023255816, 825.239263803681, 640.98..."
296,./data/data/source_data/frame/Hole_distortion\...,1,프레임,"[[894.6787830634457, 346.2404546430753, 130.61..."
297,./data/data/source_data/frame/Hole_distortion\...,1,프레임,"[[1384.1162790697674, 896.0, 85.3953488372093,..."
298,./data/data/source_data/frame/Seam_failure\207...,1,프레임,"[[310.7441860465116, 181.03703703703692, 437.9..."


In [29]:
import pandas as pd
data_fault_bumper = data_bumper.iloc[first_zero_bumper:]
data_fault_door = pd.concat([data_door_ed.iloc[first_zero_door_ed:],
                             data_door_scratch.iloc[first_zero_door_scratch:]], ignore_index=True)
data_fault_fender = data_fender.iloc[first_zero_fender:]
data_fault_frame = pd.concat([data_frame_ed.iloc[first_zero_frame_ed:],
                              data_frame_hd.iloc[first_zero_frame_hd:], data_frame_sealf.iloc[first_zero_frame_sealf:], data_frame_seamf.iloc[first_zero_frame_seamf:]], ignore_index=True)


# 300개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_fault_random_sample = data_fault_bumper.sample(n=300, replace=False)
fender_fault_random_sample = data_fault_fender.sample(n=300, replace=False)
door_fault_random_sample = data_fault_door.sample(n=300, replace=False)
frame_fault_random_sample = data_fault_frame.sample(n=300, replace=False)
# 결과 출력
# print(bumper_fault_random_sample)
# print(len(bumper_fault_random_sample))

# print(fender_fault_random_sample)
# print(len(fender_fault_random_sample))

# print(door_fault_random_sample)
# print(len(door_fault_random_sample))

# print(frame_fault_random_sample)
# print(len(frame_fault_random_sample))

fault_data = pd.concat([bumper_fault_random_sample, fender_fault_random_sample,
                        door_fault_random_sample, frame_fault_random_sample], ignore_index=True, axis=0)

fault_data

,path,quality,part,bboxes
0,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[959.3023255813954, 694.9087491599463, 2208.1..."
1,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[1773.2558139534883, 837.981308411215, 325.58..."
2,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[2348.5275454987723, 761.9356083233772, 245.3..."
3,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[1174.418604651163, 821.3592233009709, 1337.2..."
4,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[900.2054263565893, 1135.5465116279067, 1102...."
...,...,...,...,...
1195,./data/data/source_data/frame/Sealing_failure\...,0,프레임,"[[1934.4683235861905, 857.8691139388533, 164.0..."
1196,./data/data/source_data/frame/Hole_distortion\...,0,프레임,"[[1944.7790697674425, 752.0561529271205, 335.7..."
1197,./data/data/source_data/frame/Sealing_failure\...,0,프레임,"[[1840.1860465116279, 655.5092024539878, 457.1..."
1198,./data/data/source_data/frame/Exterior_damage\...,0,프레임,"[[1728.8372093023256, 803.9520958083832, 334.0..."


In [30]:
from sklearn.model_selection import train_test_split
import pandas as pd

# y에는 라벨, x에는 나머지 특성 정보
y_ok = ok_data['quality']
y_fault = fault_data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data_ok, test_data_ok = train_test_split(
    ok_data, test_size=0.2, stratify=y_ok, random_state=60
)
train_data_fault, test_data_fault = train_test_split(
    fault_data, test_size=0.2, stratify=y_fault, random_state=60
)

# 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
train_data_80_ok, valid_data_ok = train_test_split(
    train_data_ok, test_size=0.2, stratify=train_data_ok['quality'], random_state=60)
train_data_80_fault, valid_data_fault = train_test_split(
    train_data_fault, test_size=0.2, stratify=train_data_fault['quality'], random_state=60)

# CSV로 저장
train_data_80_ok.to_csv('./data/train_data_80_ok.csv', index=False)
train_data_80_fault.to_csv('./data/train_data_80_fault.csv', index=False)
test_data_ok.to_csv('./data/test_data_ok.csv', index=False)
test_data_fault.to_csv('./data/test_data_fault.csv', index=False)
valid_data_ok.to_csv('./data/valid_data_ok.csv', index=False)
valid_data_fault.to_csv('./data/valid_data_fault.csv', index=False)

# 결과 출력
print(f"y_ok 총 데이터 개수: {len(y_ok)}")
print(f"y_fault 총 데이터 개수: {len(y_fault)}")
print(f"train_data_80_ok 학습 데이터 개수: {len(train_data_80_ok)}")
print(f"train_data_80_fault 학습 데이터 개수: {len(train_data_80_fault)}")
print(f"test_data_ok 테스트 데이터 개수: {len(test_data_ok)}")
print(f"test_data_fault 테스트 데이터 개수: {len(test_data_fault)}")
print(f"valid_data_ok 검증 데이터 개수: {len(valid_data_ok)}")
print(f"valid_data_fault 검증 데이터 개수: {len(valid_data_fault)}")

y_ok 총 데이터 개수: 300
y_fault 총 데이터 개수: 1200
train_data_80_ok 학습 데이터 개수: 192
train_data_80_fault 학습 데이터 개수: 768
test_data_ok 테스트 데이터 개수: 60
test_data_fault 테스트 데이터 개수: 240
valid_data_ok 검증 데이터 개수: 48
valid_data_fault 검증 데이터 개수: 192


In [32]:
# 이미지 복사
import shutil

def copy_images(data, output_dir):
    image_dir = os.path.join(output_dir, 'images')
    os.makedirs(image_dir, exist_ok=True)

    for index, row in data.iterrows():
        image_path = row['path']
        shutil.copy(image_path, image_dir)


# 학습 데이터와 검증 데이터를 각각 처리
copy_images(train_data_80_ok, './train')
copy_images(test_data_ok, './test')
copy_images(valid_data_ok,'./valid')
copy_images(train_data_80_fault, './train')
copy_images(test_data_fault, './test')
copy_images(valid_data_fault,'./valid')

In [33]:
import ast

def parse_bboxes(data):
    """
    문자열 형태의 bounding box 정보를 파싱하여 리스트 형태로 변환하는 함수

    Args:
        data: 파싱할 데이터셋

    Returns:
        list: 파싱된 bounding box 리스트
    """

    bboxes_list = []
    for bbox_str in data["bboxes"]:
        try:
            bboxes = ast.literal_eval(bbox_str)
            bbox_list = []
            for bbox in bboxes:
                x, y, w, h = bbox
                bbox_list.append([x, y, w, h])
            bboxes_list.append(bbox_list)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox: {e}")
    return bboxes_list

# 데이터셋별 bounding box 파싱
train_ok_bboxes = parse_bboxes(train_data_80_ok)
valid_ok_bboxes = parse_bboxes(valid_data_ok)
test_ok_bboxes = parse_bboxes(test_data_ok)
train_fault_bboxes = parse_bboxes(train_data_80_fault)
valid_fault_bboxes = parse_bboxes(valid_data_fault)
test_fault_bboxes = parse_bboxes(test_data_fault)

# 결과 출력 (예시)
print(train_ok_bboxes[:5])
print(valid_ok_bboxes[:5])
print(test_ok_bboxes[:5])
print(train_fault_bboxes[:5])
print(valid_fault_bboxes[:5])
print(test_fault_bboxes[:5])

[[], [], [], [[636.9069767441861, 1294.2222222222222, 96.06976744186045, 174.22222222222223]], []]
[[], [], [], [[1412.5813953488373, 1326.2222222222222, 60.48837209302325, 462.2222222222222]], []]
[[[1330.744186046512, 1493.3333333333335, 113.86046511627907, 241.77777777777777]], [], [], [], []]
[[[955.2558139534883, 527.4418604651163, 392.651162790698, 140.65116279069764], [35.16279069767442, 902.5116279069765, 3697.9534883720917, 884.9302325581399], [972.8372093023255, 632.9302325581396, 1576.4651162790701, 234.4186046511628]], [[1881.2093023255813, 827.4251497005988, 275.4418604651163, 252.33532934131733]], [[334.04651162790697, 604.4311377245511, 492.27906976744174, 457.72455089820346]], [[1558.1395348837214, 1244.1860465116279, 191.86046511627865, 133.7209302325582], [1976.7441860465117, 1284.8837209302326, 668.6046511627903, 232.55813953488382], [3093.0232558139533, 2412.7906976744184, 430.2325581395353, 197.67441860465124]], [[1529.5813953488375, 518.1047953216374, 936.96948922

In [34]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_ok_image_paths = train_data_80_ok.iloc[:, 0].tolist()
valid_ok_image_paths = valid_data_ok.iloc[:, 0].tolist()
test_ok_image_paths = test_data_ok.iloc[:, 0].tolist()
train_fault_image_paths = train_data_80_fault.iloc[:, 0].tolist()
valid_fault_image_paths = valid_data_fault.iloc[:, 0].tolist()
test_fault_image_paths = test_data_fault.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes_ok, train_dirs_ok, train_names_ok = extract_image_info(train_ok_image_paths)
valid_sizes_ok, valid_dirs_ok, valid_names_ok = extract_image_info(valid_ok_image_paths)
test_sizes_ok, test_dirs_ok, test_names_ok = extract_image_info(test_ok_image_paths)
train_sizes_fault, train_dirs_fault, train_names_fault = extract_image_info(train_fault_image_paths)
valid_sizes_fault, valid_dirs_fault, valid_names_fault = extract_image_info(valid_fault_image_paths)
test_sizes_fault, test_dirs_fault, test_names_fault = extract_image_info(test_fault_image_paths)

# 결과 출력
# ok
print("train_sizes_ok 이미지 크기:", train_sizes_ok[:5])
print("train_dirs_ok 이미지 디렉토리:", train_dirs_ok[:5])
print("train_names_ok 파일 이름:", train_names_ok[:5])
print()
print("valid_sizes_ok 이미지 크기:", valid_sizes_ok[:5])
print("valid_dirs_ok 이미지 디렉토리:", valid_dirs_ok[:5])
print("valid_names_ok 파일 이름:", valid_names_ok[:5])
print()
print("test_sizes_ok 이미지 크기:", test_sizes_ok[:5])
print("test_dirs_ok 이미지 디렉토리:", test_dirs_ok[:5])
print("test_names_ok 파일 이름:", test_names_ok[:5])
# fault
print("train_sizes_fault 이미지 크기:", train_sizes_fault[:5])
print("train_dirs_fault 이미지 디렉토리:", train_dirs_fault[:5])
print("train_names_fault 파일 이름:", train_names_fault[:5])
print()
print("valid_sizes_fault 이미지 크기:", valid_sizes_fault[:5])
print("valid_dirs_fault 이미지 디렉토리:", valid_dirs_fault[:5])
print("valid_names_fault 파일 이름:", valid_names_fault[:5])
print()
print("test_sizes_fault 이미지 크기:", test_sizes_fault[:5])
print("test_dirs_fault 이미지 디렉토리:", test_dirs_fault[:5])
print("test_names_fault 파일 이름:", test_names_fault[:5])

train_sizes_ok 이미지 크기: [(4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000)]
train_dirs_ok 이미지 디렉토리: ['./data/data/source_data/bumper/scratch\\', './data/data/source_data/bumper/scratch\\', './data/data/source_data/fender/Exterior_damage\\', './data/data/source_data/frame/Hole_distortion\\', './data/data/source_data/door/scratch\\']
train_names_ok 파일 이름: ['205_101_10_1419f342-b811-454f-a371-57c02c6343c8', '205_101_10_60da5008-10b2-40b9-a0f6-e7dc22d2f11c', '209_102_10_319c5ffd-6e52-4711-a74f-bee9f0431a4f', '207_213_10_53b0c013-55a0-46c0-abbf-4032622525d4', '204_101_10_08e9e2fe-6870-4b41-98b9-1812b886e1e6']

valid_sizes_ok 이미지 크기: [(4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000)]
valid_dirs_ok 이미지 디렉토리: ['./data/data/source_data/fender/Exterior_damage\\', './data/data/source_data/bumper/scratch\\', './data/data/source_data/door/scratch\\', './data/data/source_data/frame/Seam_failure\\', './data/data/source_data/fender/Exterior_damage\\']
valid_names_

In [35]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 0
        elif status == 0:
            return 2
    elif part == "휀더":
        if status == 1:
            return 0
        elif status == 0:
            return 3
    elif part == "프레임":
        if status == 1:
            return 0
        elif status == 0:
            return 4
    return None  # 기본적으로 반환할 값이 없으면 None

# 예시 부품과 상태
parts_and_status_train_ok = []
parts_and_status_valid_ok = []
parts_and_status_test_ok = []
for quality, part in zip(train_data_80_ok.iloc[:, 1], train_data_80_ok.iloc[:, 2]):
    parts_and_status_train_ok.append({"part": part, "status": quality})

for quality, part in zip(test_data_ok.iloc[:, 1], test_data_ok.iloc[:, 2]):
    parts_and_status_test_ok.append({"part": part, "status": quality})

for quality, part in zip(valid_data_ok.iloc[:, 1], valid_data_ok.iloc[:, 2]):
    parts_and_status_valid_ok.append({"part": part, "status": quality})

parts_and_status_train_fault = []
parts_and_status_valid_fault = []
parts_and_status_test_fault = []
for quality, part in zip(train_data_80_fault.iloc[:, 1], train_data_80_fault.iloc[:, 2]):
    parts_and_status_train_fault.append({"part": part, "status": quality})

for quality, part in zip(test_data_fault.iloc[:, 1], test_data_fault.iloc[:, 2]):
    parts_and_status_test_fault.append({"part": part, "status": quality})

for quality, part in zip(valid_data_fault.iloc[:, 1], valid_data_fault.iloc[:, 2]):
    parts_and_status_valid_fault.append({"part": part, "status": quality})

print(f"parts_and_status_train : {parts_and_status_train_ok}")
print(f"parts_and_status_test : {parts_and_status_test_ok}")
print(f"parts_and_status_valid : {parts_and_status_valid_ok}")

print(f"parts_and_status_train : {parts_and_status_train_fault}")
print(f"parts_and_status_test : {parts_and_status_test_fault}")
print(f"parts_and_status_valid : {parts_and_status_valid_fault}")

# 라벨링 결과를 저장할 리스트
labels_train_ok = []
labels_valid_ok = []
label_test_ok = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_ok:
    label = get_label(item["part"], item["status"])
    labels_train_ok.append(label)
for item in parts_and_status_test_ok:
    label = get_label(item["part"], item["status"])
    labels_valid_ok.append(label)
for item in parts_and_status_valid_ok:
    label = get_label(item["part"], item["status"])
    label_test_ok.append(label)

# # 결과 출력
print("labels_train_ok 라벨링 결과 :", labels_train_ok)
print("labels_valid_ok 라벨링 결과 :", labels_valid_ok)
print("label_test_ok 라벨링 결과 :", label_test_ok)

# 라벨링 결과를 저장할 리스트
labels_train_fault = []
labels_valid_fault = []
label_test_fualt = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_fault:
    label = get_label(item["part"], item["status"])
    labels_train_fault.append(label)
for item in parts_and_status_test_fault:
    label = get_label(item["part"], item["status"])
    labels_valid_fault.append(label)
for item in parts_and_status_valid_fault:
    label = get_label(item["part"], item["status"])
    label_test_fualt.append(label)

# # 결과 출력
print("labels_train_fault 라벨링 결과 :", labels_train_fault)
print("labels_valid_fault 라벨링 결과 :", labels_valid_fault)
print("label_test_fualt 라벨링 결과 :", label_test_fualt)

parts_and_status_train : [{'part': '범퍼', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': 

In [36]:
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []  # 각 이미지에 대한 라벨을 담을 리스트

    if not bbox:
        # bbox가 비어있으면 기본값으로 라벨을 설정
        yolo_labels.append(f"{label} 0.0 0.0 0.0 0.0")
    else:
        # bbox가 있으면 여러 개의 파손 부위를 처리
        for x_min, y_min, width, height in bbox:
            # YOLO 형식에 맞게 좌표를 정규화
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            # 정규화된 좌표와 함께 라벨을 저장
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {
                               norm_width:.6f} {norm_height:.6f}")

    return yolo_labels


def save_yolo_labels(output_dir, bboxes, labels, size, names):
    # 각 이미지에 대해 라벨을 텍스트 파일로 저장
    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        # 라벨을 .txt 파일로 저장
        txt_file_path = os.path.join(output_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")



# 라벨을 저장할 디렉토리
train_output_dir = './train/labels'
os.makedirs(train_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(train_output_dir, train_ok_bboxes, labels_train_ok, train_sizes_ok, train_names_ok)
save_yolo_labels(train_output_dir, train_fault_bboxes, labels_train_fault, train_sizes_fault, train_names_fault)
# 라벨을 저장할 디렉토리
valid_output_dir = './valid/labels'
os.makedirs(valid_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(valid_output_dir, valid_ok_bboxes, labels_valid_ok, valid_sizes_ok, valid_names_ok)
save_yolo_labels(valid_output_dir, valid_fault_bboxes, labels_valid_fault, valid_sizes_fault, valid_names_fault)

# 라벨을 저장할 디렉토리
test_output_dir = './test/labels'
os.makedirs(test_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(test_output_dir, test_ok_bboxes, label_test_ok, test_sizes_ok, test_names_ok)
save_yolo_labels(test_output_dir, test_fault_bboxes, label_test_fualt, test_sizes_fault, test_names_fault)

In [40]:
# YAML 파일에 저장
yaml_data = {
    "train": "C:/Users/Administrator/sf_project_2/train/images",
    "test": "C:/Users/Administrator/sf_project_2/test/images",
    "val": "C:/Users/Administrator/sf_project_2/valid/images",
    "nc": 5,
    "names": ["ok",
              "bumper_faulty",
              "door_faulty",
              "fender_faulty",
              "frame_faulty"]
}

with open(f"data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

# print("YAML 파일이 생성되었습니다.")

In [ ]:
import os
from ultralytics import YOLO

# 데이터 설정 파일 경로
data_yaml = "data.yaml"

# YOLO 모델 정의 (랜덤 초기화로 학습 시작)

# n-s-m-l-x
model = YOLO("yolo11n.pt")  # 'yolov5n'은 작은 모델, 필요에 따라 yolov5s, yolov5m 선택 가능
# model.model = model.model.reset_parameters()  # 가중치 초기화 (랜덤 가중치)

# 학습 설정
model.train(
    data=data_yaml,           # 데이터 설정 파일 경로
    epochs=30,               # 학습 에폭 수
    batch=16,                 # 배치 크기
    imgsz=640,                # 입력 이미지 크기
    device=0,                 # GPU 사용
    workers=4,                # 데이터 로드 병렬 처리 워커 수
    project="C:/Users/Administrator/sf_project_2",     # 결과 저장 디렉토리
    name="part_damage_detection"       # 실험 이름
)

model.save('model_11_n_30_16_960.pt')

New https://pypi.org/project/ultralytics/8.3.65 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.64  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=C:/Users/Administrator/sf_project_2, name=part_damage_detection3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning C:\Users\Administrator\sf_project_2\train\labels... 960 images, 0 backgrounds, 61 corrupt: 100%|██████████| 960/960 [00:01<00:00, 804.33it/s] 

train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_0a40b0f8-a35e-4b1b-af34-7170be771c47.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.073217]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_0ea09e5e-c828-41a5-b78f-66d23b4f4034.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.369884]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_101061f4-1ec6-4956-9e7c-5cb648fe55ce.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.104698]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_1b2c4e6d-ccbe-4844-9573-b9f597e8d5ec.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.103913]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_29fdd5e8-cf3d-471f-9812-b2db251c876e.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordina


val: Scanning C:\Users\Administrator\sf_project_2\valid\labels... 117 images, 0 backgrounds, 13 corrupt:  49%|████▉     | 117/240 [00:00<00:00, 1165.31it/s]

In [ ]:

from ultralytics import YOLO

# 저장된 모델 불러오기
model = YOLO("model_11_n_30_16_960.pt")

# 추론
results = model.predict(source="C:/Users/Administrator/sf_project_2/test/images", imgsz=640)


image 1/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_0ef88ead-0d7d-4230-8bb1-7d49c063ae3f.jpg: 320x640 (no detections), 41.1ms
image 2/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_1bddf87a-2fec-45ab-9021-c9c6051939e0.jpg: 320x640 1 door_faulty, 25.4ms
image 3/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_22030eca-f5f3-4fc4-9f57-3460d45273c6.jpg: 320x640 (no detections), 16.7ms
image 4/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_63ee21f2-8e2f-436e-8065-1638d12bc38a.jpg: 320x640 (no detections), 16.6ms
image 5/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_66e438da-99ad-4ce7-bc95-96033bc6d2db.jpg: 320x640 (no detections), 18.5ms
image 6/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_74c86cc8-e672-4c38-bb9f-333ae1504582.jpg: 320x640 (no detections), 17.0ms
image 7/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_921c47c8-9786-4838-97ca-74aad33584a7.jpg: 320x640 (no detect

In [22]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

pred = []

for result in results:
    if len(result.boxes) > 0:  # Detection이 있을 경우
        # 각 box에서 confidence가 가장 높은 클래스 선택
        pred.append(int(result.boxes[0].cls))
    else:
        pred.append(0)  # No Detection은 '0'으로 처리


# y_test 생성
y_test = []

for i in range(len(test_test_20)):
    if test_test_20.iloc[i, 1] == 0:  # 열 1에 접근
        y_test.append(0)
    elif test_test_20.iloc[i, 2] == '범퍼':  # 열 2에 접근
        y_test.append(1)
    elif test_test_20.iloc[i, 2] == '도어':
        y_test.append(2)
    elif test_test_20.iloc[i, 2] == '휀더':
        y_test.append(3)
    elif test_test_20.iloc[i, 2] == '프레임':
        y_test.append(4)


## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt


# 위의 ROC 커브 코드 재사용 가능

print(y_test)
print(pred)

# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat, sep='\n')


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도:', accuracy)

# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred, average='macro')
print('정밀도: ', precision, sep='/n')


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred, average='macro')
print('재현율: ', recall)

[0, 0, 2, 1, 0, 0, 2, 0, 4, 0, 0, 0, 0, 0, 4, 0, 1, 4, 0, 1, 3, 0, 0, 0, 1, 0, 2, 3, 0, 2, 4, 0, 4, 4, 1, 4, 0, 0, 4, 0, 1, 0, 4, 4, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 4, 0, 0, 2, 0, 4, 0, 0, 0, 0, 2, 0, 0, 1, 0, 4, 0, 2, 3, 0, 1, 0, 0, 4, 1, 2, 0, 4, 4, 4, 2, 4, 0, 4, 0, 4, 0, 4, 0, 1, 1, 0, 4, 0, 0, 4, 0, 4, 4, 4, 4, 0, 0, 1, 1, 0, 2, 4, 0, 1, 0, 3, 4, 0, 1, 0, 0, 4, 2, 1, 0, 3, 4, 0, 1, 0, 0, 3, 1, 0, 1, 0, 0, 3, 0, 0, 4, 2, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 3, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 1, 4, 2, 0, 3, 0, 0, 4, 0, 0, 0, 2, 4, 0, 0, 1, 0, 0, 0, 0, 4, 2, 2, 0, 4, 4, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 4, 0, 0, 4, 0, 0, 3, 1, 4, 0, 2, 4, 0, 0, 4, 4, 0, 3, 4, 0, 0, 0, 0, 3, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 4, 2, 0, 3, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 4, 0, 0, 0, 0, 4, 0, 2, 4, 0, 0, 4, 0, 1, 3, 3, 2]
[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 2, 1, 1, 0, 2, 2, 2, 1, 2, 0, 2, 0, 0, 0,